In [ ]:
"""entonces es simplemente dado un estado y lo que se hizo en el estado para aumentar 
el score , luego es hacer factor comunt entre los estados y sus outputs para crear una
estructura equivalente.Pero entonces como hace para saber cual output seleccionar en
primer lieu ? c'est aleatoire s'il ne connait pas l'etat actuel ,apres tu pourra faire
la comparaison d'un nouvel etat par rapport a un autre connu en comparant le nombre de 
bits egaux de l'etat actuel"""

In [2]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [37]:
import pygame
import random

# Constants
GRID_SIZE = 20
MATRIX_WIDTH = 40
MATRIX_HEIGHT = 15
WIDTH, HEIGHT = 800, 600
MATRIX_HEIGHT_PX = HEIGHT // 2

# Initialize pygame
pygame.init()

# Screen setup
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Game and Matrix Window")

# Colors
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BRIGHT_GREEN = (150, 255, 150)  # For spawn effect
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)

# Game objects
PLAYER_WIDTH, PLAYER_HEIGHT = 40, 40
FOOD_SIZE = 5
player1 = pygame.Rect(WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)
player2 = pygame.Rect(3 * WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)

# Food system
class Food:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)
        self.spawn_effect = 15  # Frames the spawn effect will last

food_list = [Food(random.randint(0, WIDTH - FOOD_SIZE), 
                 random.randint(0, HEIGHT // 2 - FOOD_SIZE)) for _ in range(5)]

# Player states
player1_life = 10
player2_life = 10
steal_cooldown = 0
STEAL_COOLDOWN_MAX = 45
steal_effect_timer = 0

# Fonts
font = pygame.font.Font(None, 36)

def spawn_new_food():
    return Food(random.randint(0, WIDTH - FOOD_SIZE),
                random.randint(0, HEIGHT // 2 - FOOD_SIZE))

def update_matrix():
    game_matrix = [[0] * MATRIX_WIDTH for _ in range(MATRIX_HEIGHT)]
    for player in [player1, player2]:
        for y in range(player.top // GRID_SIZE, (player.bottom) // GRID_SIZE):
            for x in range(player.left // GRID_SIZE, (player.right) // GRID_SIZE):
                if 0 <= y < MATRIX_HEIGHT and 0 <= x < MATRIX_WIDTH:
                    game_matrix[y][x] = 1
    for food in food_list:
        row, col = food.rect.top // GRID_SIZE, food.rect.left // GRID_SIZE
        if 0 <= row < MATRIX_HEIGHT and 0 <= col < MATRIX_WIDTH:
            game_matrix[row][col] = 1
    return game_matrix

def draw_matrix(game_matrix):
    matrix_surface = pygame.Surface((WIDTH, MATRIX_HEIGHT_PX))
    matrix_surface.fill(BLACK)
    for y, row in enumerate(game_matrix):
        row_text = " ".join(str(cell) for cell in row)
        text_surface = font.render(row_text, True, WHITE)
        matrix_surface.blit(text_surface, (10, y * 20))
    screen.blit(matrix_surface, (0, HEIGHT // 2))

def draw_game():
    screen.fill(BLACK)
    
    # Draw players with steal effect
    if steal_effect_timer > 0:
        pygame.draw.rect(screen, YELLOW, player1)
        pygame.draw.rect(screen, YELLOW, player2)
    else:
        pygame.draw.rect(screen, BLUE, player1)
        pygame.draw.rect(screen, RED, player2)
    
    # Draw food with spawn effect
    for food in food_list:
        if food.spawn_effect > 0:
            # Draw growing effect
            effect_size = FOOD_SIZE + food.spawn_effect
            effect_rect = pygame.Rect(
                food.rect.centerx - effect_size//2,
                food.rect.centery - effect_size//2,
                effect_size,
                effect_size
            )
            pygame.draw.rect(screen, BRIGHT_GREEN, effect_rect, 2)
            pygame.draw.rect(screen, GREEN, food.rect)
            food.spawn_effect -= 1
        else:
            pygame.draw.rect(screen, GREEN, food.rect)
    
    life_text1 = font.render(f"Player 1 Life: {player1_life}", True, WHITE)
    life_text2 = font.render(f"Player 2 Life: {player2_life}", True, WHITE)
    screen.blit(life_text1, (10, 10))
    screen.blit(life_text2, (WIDTH - 200, 10))

def handle_movement():
    global player1, player2
    keys = pygame.key.get_pressed()
    move_speed = 5

    # Store old positions for collision resolution
    old_pos1 = player1.copy()
    old_pos2 = player2.copy()

    # Player 1 movement
    if keys[pygame.K_w] and player1.top > 0:
        player1.y -= move_speed
    if keys[pygame.K_s] and player1.bottom < HEIGHT // 2:
        player1.y += move_speed
    if keys[pygame.K_a] and player1.left > 0:
        player1.x -= move_speed
    if keys[pygame.K_d] and player1.right < WIDTH:
        player1.x += move_speed

    # Player 2 movement
    if keys[pygame.K_UP] and player2.top > 0:
        player2.y -= move_speed
    if keys[pygame.K_DOWN] and player2.bottom < HEIGHT // 2:
        player2.y += move_speed
    if keys[pygame.K_LEFT] and player2.left > 0:
        player2.x -= move_speed
    if keys[pygame.K_RIGHT] and player2.right < WIDTH:
        player2.x += move_speed

    # Check player collision
    if player1.colliderect(player2):
        player1.x, player1.y = old_pos1.x, old_pos1.y
        player2.x, player2.y = old_pos2.x, old_pos2.y

    # Check food collisions (if not trying to eat)
    for food in food_list:
        if player1.colliderect(food.rect) and not keys[pygame.K_e]:
            player1.x, player1.y = old_pos1.x, old_pos1.y
        if player2.colliderect(food.rect) and not keys[pygame.K_r]:
            player2.x, player2.y = old_pos2.x, old_pos2.y

def handle_steal():
    global player1_life, player2_life, steal_cooldown, steal_effect_timer
    
    if steal_cooldown > 0:
        steal_cooldown -= 1
        return

    keys = pygame.key.get_pressed()
    
    steal_range = pygame.Rect(player1.x - 10, player1.y - 10, 
                             PLAYER_WIDTH + 20, PLAYER_HEIGHT + 20)
    
    if steal_range.colliderect(player2):
        if keys[pygame.K_SPACE] and player2_life > 0:
            player2_life -= 1
            player1_life += 1
            steal_cooldown = STEAL_COOLDOWN_MAX
            steal_effect_timer = 15
            print("Player 1 stole a life! P1:", player1_life, "P2:", player2_life)
            
        if keys[pygame.K_RETURN] and player1_life > 0:
            player1_life -= 1
            player2_life += 1
            steal_cooldown = STEAL_COOLDOWN_MAX
            steal_effect_timer = 15
            print("Player 2 stole a life! P1:", player1_life, "P2:", player2_life)

def handle_food():
    global food_list, player1_life, player2_life
    
    keys = pygame.key.get_pressed()
    
    p1_food_detect = pygame.Rect(player1.x - 5, player1.y - 5, 
                                PLAYER_WIDTH + 10, PLAYER_HEIGHT + 10)
    p2_food_detect = pygame.Rect(player2.x - 5, player2.y - 5, 
                                PLAYER_WIDTH + 10, PLAYER_HEIGHT + 10)
    
    for food in food_list[:]:
        if p1_food_detect.colliderect(food.rect) and keys[pygame.K_e]:
            food_list.remove(food)
            player1_life += 1
            print("Player 1 ate food! Life:", player1_life)
            food_list.append(spawn_new_food())
            
        if p2_food_detect.colliderect(food.rect) and keys[pygame.K_r]:
            food_list.remove(food)
            player2_life += 1
            print("Player 2 ate food! Life:", player2_life)
            food_list.append(spawn_new_food())

# Game loop
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    handle_movement()
    handle_steal()
    handle_food()
    
    if steal_effect_timer > 0:
        steal_effect_timer -= 1
    
    game_matrix = update_matrix()
    draw_game()
    draw_matrix(game_matrix)

    if player1_life <= 0 or player2_life <= 0:
        running = False

    pygame.display.update()
    clock.tick(60)

pygame.quit()

Player 2 ate food! Life: 11
Player 2 ate food! Life: 12
Player 2 ate food! Life: 13
Player 2 ate food! Life: 14
Player 2 ate food! Life: 15
Player 1 stole a life! P1: 11 P2: 14
Player 1 stole a life! P1: 12 P2: 13
Player 1 stole a life! P1: 13 P2: 12
Player 2 stole a life! P1: 12 P2: 13
Player 2 stole a life! P1: 11 P2: 14
Player 2 stole a life! P1: 10 P2: 15
Player 2 stole a life! P1: 9 P2: 16
Player 2 stole a life! P1: 8 P2: 17
Player 2 stole a life! P1: 7 P2: 18
Player 2 stole a life! P1: 6 P2: 19
Player 2 stole a life! P1: 5 P2: 20
Player 2 ate food! Life: 21
Player 2 ate food! Life: 22
Player 2 ate food! Life: 23
Player 2 ate food! Life: 24
Player 2 ate food! Life: 25
Player 2 ate food! Life: 26
Player 2 ate food! Life: 27
Player 2 ate food! Life: 28
Player 2 ate food! Life: 29
Player 2 ate food! Life: 30
Player 2 ate food! Life: 31
Player 2 ate food! Life: 32
Player 2 ate food! Life: 33
Player 2 ate food! Life: 34
Player 2 ate food! Life: 35


In [8]:
import pygame
import random

# Constants
GRID_SIZE = 20
MATRIX_WIDTH = 40
MATRIX_HEIGHT = 15
WIDTH, HEIGHT = 800, 600
MATRIX_HEIGHT_PX = HEIGHT // 2

# Initialize pygame
pygame.init()

# Screen setup
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Game and Matrix Window")

# Colors
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BRIGHT_GREEN = (150, 255, 150)  # For spawn effect
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)

# Game objects
PLAYER_WIDTH, PLAYER_HEIGHT = 40, 40
FOOD_SIZE = 5
player1 = pygame.Rect(WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)
player2 = pygame.Rect(3 * WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)

# Food system
class Food:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)
        self.spawn_effect = 15  # Frames the spawn effect will last

food_list = [Food(random.randint(0, WIDTH - FOOD_SIZE), 
                 random.randint(0, HEIGHT // 2 - FOOD_SIZE)) for _ in range(5)]

# Player states
player1_life = 10
player2_life = 10
steal_cooldown = 0
STEAL_COOLDOWN_MAX = 45
steal_effect_timer = 0

# Fonts
font = pygame.font.Font(None, 36)

def spawn_new_food():
    return Food(random.randint(0, WIDTH - FOOD_SIZE),
                random.randint(0, HEIGHT // 2 - FOOD_SIZE))

def update_matrix():
    game_matrix = [[0] * MATRIX_WIDTH for _ in range(MATRIX_HEIGHT)]
    for player in [player1, player2]:
        for y in range(player.top // GRID_SIZE, (player.bottom) // GRID_SIZE):
            for x in range(player.left // GRID_SIZE, (player.right) // GRID_SIZE):
                if 0 <= y < MATRIX_HEIGHT and 0 <= x < MATRIX_WIDTH:
                    game_matrix[y][x] = 1
    for food in food_list:
        row, col = food.rect.top // GRID_SIZE, food.rect.left // GRID_SIZE
        if 0 <= row < MATRIX_HEIGHT and 0 <= col < MATRIX_WIDTH:
            game_matrix[row][col] = 1
    return game_matrix

def draw_matrix(game_matrix):
    matrix_surface = pygame.Surface((WIDTH, MATRIX_HEIGHT_PX))
    matrix_surface.fill(BLACK)
    for y, row in enumerate(game_matrix):
        row_text = " ".join(str(cell) for cell in row)
        text_surface = font.render(row_text, True, WHITE)
        matrix_surface.blit(text_surface, (10, y * 20))
    screen.blit(matrix_surface, (0, HEIGHT // 2))

def draw_game():
    screen.fill(BLACK)
    
    # Draw players with steal effect
    if steal_effect_timer > 0:
        pygame.draw.rect(screen, YELLOW, player1)
        pygame.draw.rect(screen, YELLOW, player2)
    else:
        pygame.draw.rect(screen, BLUE, player1)
        pygame.draw.rect(screen, RED, player2)
    
    # Draw food with spawn effect
    for food in food_list:
        if food.spawn_effect > 0:
            # Draw growing effect
            effect_size = FOOD_SIZE + food.spawn_effect
            effect_rect = pygame.Rect(
                food.rect.centerx - effect_size//2,
                food.rect.centery - effect_size//2,
                effect_size,
                effect_size
            )
            pygame.draw.rect(screen, BRIGHT_GREEN, effect_rect, 2)
            pygame.draw.rect(screen, GREEN, food.rect)
            food.spawn_effect -= 1
        else:
            pygame.draw.rect(screen, GREEN, food.rect)
    
    life_text1 = font.render(f"Player 1 Life: {player1_life}", True, WHITE)
    life_text2 = font.render(f"Player 2 Life: {player2_life}", True, WHITE)
    screen.blit(life_text1, (10, 10))
    screen.blit(life_text2, (WIDTH - 200, 10))

def handle_movement():
    global player1, player2
    keys = pygame.key.get_pressed()
    move_speed = 5

    # Store old positions for collision resolution
    old_pos1 = player1.copy()
    old_pos2 = player2.copy()

    # Player 1 movement
    if keys[pygame.K_w] and player1.top > 0:
        player1.y -= move_speed
    if keys[pygame.K_s] and player1.bottom < HEIGHT // 2:
        player1.y += move_speed
    if keys[pygame.K_a] and player1.left > 0:
        player1.x -= move_speed
    if keys[pygame.K_d] and player1.right < WIDTH:
        player1.x += move_speed

    # Player 2 movement
    if keys[pygame.K_UP] and player2.top > 0:
        player2.y -= move_speed
    if keys[pygame.K_DOWN] and player2.bottom < HEIGHT // 2:
        player2.y += move_speed
    if keys[pygame.K_LEFT] and player2.left > 0:
        player2.x -= move_speed
    if keys[pygame.K_RIGHT] and player2.right < WIDTH:
        player2.x += move_speed

    # Check player collision
    if player1.colliderect(player2):
        player1.x, player1.y = old_pos1.x, old_pos1.y
        player2.x, player2.y = old_pos2.x, old_pos2.y

    # Check food collisions (if not trying to eat)
    for food in food_list:
        if player1.colliderect(food.rect) and not keys[pygame.K_e]:
            player1.x, player1.y = old_pos1.x, old_pos1.y
        if player2.colliderect(food.rect) and not keys[pygame.K_r]:
            player2.x, player2.y = old_pos2.x, old_pos2.y

def handle_steal():
    global player1_life, player2_life, steal_cooldown, steal_effect_timer
    
    if steal_cooldown > 0:
        steal_cooldown -= 1
        return

    keys = pygame.key.get_pressed()
    
    steal_range = pygame.Rect(player1.x - 10, player1.y - 10, 
                             PLAYER_WIDTH + 20, PLAYER_HEIGHT + 20)
    
    if steal_range.colliderect(player2):
        if keys[pygame.K_SPACE] and player2_life > 0:
            player2_life -= 1
            player1_life += 1
            steal_cooldown = STEAL_COOLDOWN_MAX
            steal_effect_timer = 15
            print("Player 1 stole a life! P1:", player1_life, "P2:", player2_life)
            
        if keys[pygame.K_RETURN] and player1_life > 0:
            player1_life -= 1
            player2_life += 1
            steal_cooldown = STEAL_COOLDOWN_MAX
            steal_effect_timer = 15
            print("Player 2 stole a life! P1:", player1_life, "P2:", player2_life)

def handle_food():
    global food_list, player1_life, player2_life
    
    keys = pygame.key.get_pressed()
    
    p1_food_detect = pygame.Rect(player1.x - 5, player1.y - 5, 
                                PLAYER_WIDTH + 10, PLAYER_HEIGHT + 10)
    p2_food_detect = pygame.Rect(player2.x - 5, player2.y - 5, 
                                PLAYER_WIDTH + 10, PLAYER_HEIGHT + 10)
    
    for food in food_list[:]:
        if p1_food_detect.colliderect(food.rect) and keys[pygame.K_e]:
            food_list.remove(food)
            player1_life += 1
            print("Player 1 ate food! Life:", player1_life)
            food_list.append(spawn_new_food())
            
        if p2_food_detect.colliderect(food.rect) and keys[pygame.K_r]:
            food_list.remove(food)
            player2_life += 1
            print("Player 2 ate food! Life:", player2_life)
            food_list.append(spawn_new_food())

# Game loop
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    handle_movement()
    handle_steal()
    handle_food()
    
    if steal_effect_timer > 0:
        steal_effect_timer -= 1
    
    game_matrix = update_matrix()
    draw_game()
    draw_matrix(game_matrix)

    if player1_life <= 0 or player2_life <= 0:
        running = False

    pygame.display.update()
    clock.tick(60)

pygame.quit()


Player 2 ate food! Life: 11
Player 1 stole a life! P1: 11 P2: 10
Player 1 stole a life! P1: 12 P2: 9
Player 1 stole a life! P1: 13 P2: 8
Player 1 stole a life! P1: 14 P2: 7
Player 1 stole a life! P1: 15 P2: 6
Player 1 stole a life! P1: 16 P2: 5
Player 2 ate food! Life: 6
Player 2 ate food! Life: 7
Player 2 stole a life! P1: 15 P2: 8
Player 2 stole a life! P1: 14 P2: 9
Player 2 stole a life! P1: 13 P2: 10
Player 2 stole a life! P1: 12 P2: 11
Player 2 stole a life! P1: 11 P2: 12
Player 2 stole a life! P1: 10 P2: 13


In [12]:
import pygame
import random

# Constants
GRID_SIZE = 20
MATRIX_WIDTH = 40
MATRIX_HEIGHT = 15
WIDTH, HEIGHT = 800, 600
MATRIX_HEIGHT_PX = HEIGHT // 2

# Initialize pygame
pygame.init()

# Screen setup
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Game and Matrix Window")

# Colors
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BRIGHT_GREEN = (150, 255, 150)  # For spawn effect
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)

# Game objects
PLAYER_WIDTH, PLAYER_HEIGHT = 40, 40
FOOD_SIZE = 5
player1 = pygame.Rect(WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)
player2 = pygame.Rect(3 * WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)

# Food system
class Food:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)
        self.spawn_effect = 15  # Frames the spawn effect will last

food_list = [Food(random.randint(0, WIDTH - FOOD_SIZE), 
                 random.randint(0, HEIGHT // 2 - FOOD_SIZE)) for _ in range(5)]

# Player states
player1_life = 10
player2_life = 10
steal_cooldown = 0
STEAL_COOLDOWN_MAX = 45
steal_effect_timer = 0

# Fonts
font = pygame.font.Font(None, 36)

def spawn_new_food():
    return Food(random.randint(0, WIDTH - FOOD_SIZE),
                random.randint(0, HEIGHT // 2 - FOOD_SIZE))

def update_matrix():
    game_matrix = [[0] * MATRIX_WIDTH for _ in range(MATRIX_HEIGHT)]
    for player in [player1, player2]:
        for y in range(player.top // GRID_SIZE, (player.bottom) // GRID_SIZE):
            for x in range(player.left // GRID_SIZE, (player.right) // GRID_SIZE):
                if 0 <= y < MATRIX_HEIGHT and 0 <= x < MATRIX_WIDTH:
                    game_matrix[y][x] = 1
    for food in food_list:
        row, col = food.rect.top // GRID_SIZE, food.rect.left // GRID_SIZE
        if 0 <= row < MATRIX_HEIGHT and 0 <= col < MATRIX_WIDTH:
            game_matrix[row][col] = 1
    return game_matrix

def draw_matrix(game_matrix):
    matrix_surface = pygame.Surface((WIDTH, MATRIX_HEIGHT_PX))
    matrix_surface.fill(BLACK)
    for y, row in enumerate(game_matrix):
        row_text = " ".join(str(cell) for cell in row)
        text_surface = font.render(row_text, True, WHITE)
        matrix_surface.blit(text_surface, (10, y * 20))
    screen.blit(matrix_surface, (0, HEIGHT // 2))

def draw_game():
    screen.fill(BLACK)
    
    # Draw players with steal effect
    if steal_effect_timer > 0:
        pygame.draw.rect(screen, YELLOW, player1)
        pygame.draw.rect(screen, YELLOW, player2)
    else:
        pygame.draw.rect(screen, BLUE, player1)
        pygame.draw.rect(screen, RED, player2)
    
    # Draw food with spawn effect
    for food in food_list:
        if food.spawn_effect > 0:
            # Draw growing effect
            effect_size = FOOD_SIZE + food.spawn_effect
            effect_rect = pygame.Rect(
                food.rect.centerx - effect_size//2,
                food.rect.centery - effect_size//2,
                effect_size,
                effect_size
            )
            pygame.draw.rect(screen, BRIGHT_GREEN, effect_rect, 2)
            pygame.draw.rect(screen, GREEN, food.rect)
            food.spawn_effect -= 1
        else:
            pygame.draw.rect(screen, GREEN, food.rect)
    
    life_text1 = font.render(f"Player 1 Life: {player1_life}", True, WHITE)
    life_text2 = font.render(f"Player 2 Life: {player2_life}", True, WHITE)
    screen.blit(life_text1, (10, 10))
    screen.blit(life_text2, (WIDTH - 200, 10))

def handle_movement(keys):
    global player1, player2
    move_speed = 5

    # Store old positions for collision resolution
    old_pos1 = player1.copy()
    old_pos2 = player2.copy()

    # Player 1 (AI) movement (Blue player)
    if keys.get('W') and player1.top > 0:
        player1.y -= move_speed
    if keys.get('S') and player1.bottom < HEIGHT // 2:
        player1.y += move_speed
    if keys.get('A') and player1.left > 0:
        player1.x -= move_speed
    if keys.get('D') and player1.right < WIDTH:
        player1.x += move_speed

    # Player 2 (Human player) movement (Red player)
    if keys.get('UP') and player2.top > 0:
        player2.y -= move_speed
    if keys.get('DOWN') and player2.bottom < HEIGHT // 2:
        player2.y += move_speed
    if keys.get('LEFT') and player2.left > 0:
        player2.x -= move_speed
    if keys.get('RIGHT') and player2.right < WIDTH:
        player2.x += move_speed

    # Collision checks (with players)
    if player1.colliderect(player2):
        player1.x, player1.y = old_pos1.x, old_pos1.y
        player2.x, player2.y = old_pos2.x, old_pos2.y

def ai_player_action():
    # Randomly pick a direction for the blue player
    move_choices = ['W', 'S', 'A', 'D']
    action = random.choice(move_choices)
    
    keys = {action: True}

    # Handle movement for blue player (AI)
    handle_movement(keys)

# Game loop
running = True
clock = pygame.time.Clock()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Handle AI movement for blue player
    ai_player_action()  # This will simulate random movements for the AI

    # Handle human player movement (check keypresses)
    keys = pygame.key.get_pressed()
    handle_movement({'UP': keys[pygame.K_UP], 'DOWN': keys[pygame.K_DOWN], 
                      'LEFT': keys[pygame.K_LEFT], 'RIGHT': keys[pygame.K_RIGHT]})

    # Draw the game
    draw_game()
    draw_matrix(update_matrix())

    if player1_life <= 0 or player2_life <= 0:
        running = False

    pygame.display.update()
    clock.tick(60)

pygame.quit()


In [14]:
import pygame
import random

# Constants
GRID_SIZE = 20
MATRIX_WIDTH = 40
MATRIX_HEIGHT = 15
WIDTH, HEIGHT = 800, 600
MATRIX_HEIGHT_PX = HEIGHT // 2

# Initialize pygame
pygame.init()

# Screen setup
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Game and Matrix Window")

# Colors
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BRIGHT_GREEN = (150, 255, 150)  # For spawn effect
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)

# Game objects
PLAYER_WIDTH, PLAYER_HEIGHT = 40, 40
FOOD_SIZE = 5
player1 = pygame.Rect(WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)
player2 = pygame.Rect(3 * WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)

# Food system
class Food:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)
        self.spawn_effect = 15  # Frames the spawn effect will last

food_list = []

def spawn_new_food():
    while True:
        new_food = Food(random.randint(0, WIDTH - FOOD_SIZE),
                        random.randint(0, HEIGHT // 2 - FOOD_SIZE))
        # Ensure food doesn't overlap with players
        if not player1.colliderect(new_food.rect) and not player2.colliderect(new_food.rect):
            return new_food

# Player states
player1_life = 10
player2_life = 10
steal_cooldown = 0
STEAL_COOLDOWN_MAX = 45
steal_effect_timer = 0

# Fonts
font = pygame.font.Font(None, 36)

def update_matrix():
    game_matrix = [[0] * MATRIX_WIDTH for _ in range(MATRIX_HEIGHT)]
    for player in [player1, player2]:
        for y in range(player.top // GRID_SIZE, (player.bottom) // GRID_SIZE):
            for x in range(player.left // GRID_SIZE, (player.right) // GRID_SIZE):
                if 0 <= y < MATRIX_HEIGHT and 0 <= x < MATRIX_WIDTH:
                    game_matrix[y][x] = 1
    for food in food_list:
        row, col = food.rect.top // GRID_SIZE, food.rect.left // GRID_SIZE
        if 0 <= row < MATRIX_HEIGHT and 0 <= col < MATRIX_WIDTH:
            game_matrix[row][col] = 1
    return game_matrix

def draw_matrix(game_matrix):
    matrix_surface = pygame.Surface((WIDTH, MATRIX_HEIGHT_PX))
    matrix_surface.fill(BLACK)
    for y, row in enumerate(game_matrix):
        row_text = " ".join(str(cell) for cell in row)
        text_surface = font.render(row_text, True, WHITE)
        matrix_surface.blit(text_surface, (10, y * 20))
    screen.blit(matrix_surface, (0, HEIGHT // 2))

def draw_game():
    screen.fill(BLACK)
    
    # Draw players with steal effect
    if steal_effect_timer > 0:
        pygame.draw.rect(screen, YELLOW, player1)
        pygame.draw.rect(screen, YELLOW, player2)
    else:
        pygame.draw.rect(screen, BLUE, player1)
        pygame.draw.rect(screen, RED, player2)
    
    # Draw food with spawn effect
    for food in food_list:
        if food.spawn_effect > 0:
            # Draw growing effect
            effect_size = FOOD_SIZE + food.spawn_effect
            effect_rect = pygame.Rect(
                food.rect.centerx - effect_size//2,
                food.rect.centery - effect_size//2,
                effect_size,
                effect_size
            )
            pygame.draw.rect(screen, BRIGHT_GREEN, effect_rect, 2)
            pygame.draw.rect(screen, GREEN, food.rect)
            food.spawn_effect -= 1
        else:
            pygame.draw.rect(screen, GREEN, food.rect)
    
    life_text1 = font.render(f"Player 1 Life: {player1_life}", True, WHITE)
    life_text2 = font.render(f"Player 2 Life: {player2_life}", True, WHITE)
    screen.blit(life_text1, (10, 10))
    screen.blit(life_text2, (WIDTH - 200, 10))

def handle_movement():
    global player1, player2
    keys = pygame.key.get_pressed()
    move_speed = 5

    # Store old positions for collision resolution
    old_pos1 = player1.copy()
    old_pos2 = player2.copy()

    # Player 1 movement
    if keys[pygame.K_w] and player1.top > 0:
        player1.y -= move_speed
    if keys[pygame.K_s] and player1.bottom < HEIGHT // 2:
        player1.y += move_speed
    if keys[pygame.K_a] and player1.left > 0:
        player1.x -= move_speed
    if keys[pygame.K_d] and player1.right < WIDTH:
        player1.x += move_speed

    # Player 2 movement
    if keys[pygame.K_UP] and player2.top > 0:
        player2.y -= move_speed
    if keys[pygame.K_DOWN] and player2.bottom < HEIGHT // 2:
        player2.y += move_speed
    if keys[pygame.K_LEFT] and player2.left > 0:
        player2.x -= move_speed
    if keys[pygame.K_RIGHT] and player2.right < WIDTH:
        player2.x += move_speed

    # Check player collision
    if player1.colliderect(player2):
        player1.x, player1.y = old_pos1.x, old_pos1.y
        player2.x, player2.y = old_pos2.x, old_pos2.y

    # Check food collisions (if not trying to eat)
    for food in food_list:
        if player1.colliderect(food.rect) and not keys[pygame.K_e]:
            player1.x, player1.y = old_pos1.x, old_pos1.y
        if player2.colliderect(food.rect) and not keys[pygame.K_r]:
            player2.x, player2.y = old_pos2.x, old_pos2.y

def handle_steal():
    global player1_life, player2_life, steal_cooldown, steal_effect_timer
    
    if steal_cooldown > 0:
        steal_cooldown -= 1
        return

    keys = pygame.key.get_pressed()
    
    steal_range = pygame.Rect(player1.x - 10, player1.y - 10, 
                             PLAYER_WIDTH + 20, PLAYER_HEIGHT + 20)
    
    if steal_range.colliderect(player2):
        if keys[pygame.K_SPACE] and player2_life > 0:
            player2_life -= 1
            player1_life += 1
            steal_cooldown = STEAL_COOLDOWN_MAX
            steal_effect_timer = 15
            print("Player 1 stole a life! P1:", player1_life, "P2:", player2_life)
            
        if keys[pygame.K_RETURN] and player1_life > 0:
            player1_life -= 1
            player2_life += 1
            steal_cooldown = STEAL_COOLDOWN_MAX
            steal_effect_timer = 15
            print("Player 2 stole a life! P1:", player1_life, "P2:", player2_life)

def handle_food():
    global food_list, player1_life, player2_life
    
    keys = pygame.key.get_pressed()
    
    p1_food_detect = pygame.Rect(player1.x - 5, player1.y - 5, 
                                PLAYER_WIDTH + 10, PLAYER_HEIGHT + 10)
    p2_food_detect = pygame.Rect(player2.x - 5, player2.y - 5, 
                                PLAYER_WIDTH + 10, PLAYER_HEIGHT + 10)
    
    for food in food_list[:]:
        if p1_food_detect.colliderect(food.rect) and keys[pygame.K_e]:
            food_list.remove(food)
            player1_life += 1
            print("Player 1 ate food! Life:", player1_life)
            food_list.append(spawn_new_food())
            
        if p2_food_detect.colliderect(food.rect) and keys[pygame.K_r]:
            food_list.remove(food)
            player2_life += 1
            print("Player 2 ate food! Life:", player2_life)
            food_list.append(spawn_new_food())

# Game loop
clock = pygame.time.Clock()
running = True

# Add initial food
food_list = [spawn_new_food() for _ in range(5)]

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    handle_movement()
    handle_steal()
    handle_food()
    
    if steal_effect_timer > 0:
        steal_effect_timer -= 1
    
    game_matrix = update_matrix()
    draw_game()
    draw_matrix(game_matrix)

    if player1_life <= 0 or player2_life <= 0:
        running = False

    pygame.display.update()
    clock.tick(60)

pygame.quit()


Player 2 ate food! Life: 11
Player 1 stole a life! P1: 11 P2: 10
Player 2 stole a life! P1: 10 P2: 11
Player 2 stole a life! P1: 9 P2: 12
Player 2 stole a life! P1: 8 P2: 13
Player 1 stole a life! P1: 9 P2: 12


In [15]:
import pygame
import random

# Constants
GRID_SIZE = 20
MATRIX_WIDTH = 40
MATRIX_HEIGHT = 15
WIDTH, HEIGHT = 800, 600
MATRIX_HEIGHT_PX = HEIGHT // 2

# Initialize pygame
pygame.init()

# Screen setup
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Game and Matrix Window")

# Colors
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BRIGHT_GREEN = (150, 255, 150)  # For spawn effect
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)

# Game objects
PLAYER_WIDTH, PLAYER_HEIGHT = 40, 40
FOOD_SIZE = 5
player1 = pygame.Rect(WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)
player2 = pygame.Rect(3 * WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)

# Food system
class Food:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)
        self.spawn_effect = 15  # Frames the spawn effect will last

food_list = []

def spawn_new_food():
    while True:
        new_food = Food(random.randint(0, WIDTH - FOOD_SIZE),
                        random.randint(0, HEIGHT // 2 - FOOD_SIZE))
        # Ensure food doesn't overlap with players
        if not player1.colliderect(new_food.rect) and not player2.colliderect(new_food.rect):
            return new_food

# Player states
player1_life = 10
player2_life = 10
steal_cooldown = 0
STEAL_COOLDOWN_MAX = 45
steal_effect_timer = 0

# Fonts
font = pygame.font.Font(None, 36)

def update_matrix():
    game_matrix = [[0] * MATRIX_WIDTH for _ in range(MATRIX_HEIGHT)]
    for player in [player1, player2]:
        for y in range(player.top // GRID_SIZE, (player.bottom) // GRID_SIZE):
            for x in range(player.left // GRID_SIZE, (player.right) // GRID_SIZE):
                if 0 <= y < MATRIX_HEIGHT and 0 <= x < MATRIX_WIDTH:
                    game_matrix[y][x] = 1
    for food in food_list:
        row, col = food.rect.top // GRID_SIZE, food.rect.left // GRID_SIZE
        if 0 <= row < MATRIX_HEIGHT and 0 <= col < MATRIX_WIDTH:
            game_matrix[row][col] = 1
    return game_matrix

def draw_matrix(game_matrix):
    matrix_surface = pygame.Surface((WIDTH, MATRIX_HEIGHT_PX))
    matrix_surface.fill(BLACK)
    for y, row in enumerate(game_matrix):
        row_text = " ".join(str(cell) for cell in row)
        text_surface = font.render(row_text, True, WHITE)
        matrix_surface.blit(text_surface, (10, y * 20))
    screen.blit(matrix_surface, (0, HEIGHT // 2))

def draw_game():
    screen.fill(BLACK)
    
    # Draw players with steal effect
    if steal_effect_timer > 0:
        pygame.draw.rect(screen, YELLOW, player1)
        pygame.draw.rect(screen, YELLOW, player2)
    else:
        pygame.draw.rect(screen, BLUE, player1)
        pygame.draw.rect(screen, RED, player2)
    
    # Draw food with spawn effect
    for food in food_list:
        if food.spawn_effect > 0:
            # Draw growing effect
            effect_size = FOOD_SIZE + food.spawn_effect
            effect_rect = pygame.Rect(
                food.rect.centerx - effect_size//2,
                food.rect.centery - effect_size//2,
                effect_size,
                effect_size
            )
            pygame.draw.rect(screen, BRIGHT_GREEN, effect_rect, 2)
            pygame.draw.rect(screen, GREEN, food.rect)
            food.spawn_effect -= 1
        else:
            pygame.draw.rect(screen, GREEN, food.rect)
    
    life_text1 = font.render(f"Player 1 Life: {player1_life}", True, WHITE)
    life_text2 = font.render(f"Player 2 Life: {player2_life}", True, WHITE)
    screen.blit(life_text1, (10, 10))
    screen.blit(life_text2, (WIDTH - 200, 10))

def handle_movement():
    global player1, player2
    keys = pygame.key.get_pressed()
    move_speed = 5

    # Store old positions for collision resolution
    old_pos1 = player1.copy()
    old_pos2 = player2.copy()

    # Player 1 movement
    if keys[pygame.K_w] and player1.top > 0:
        player1.y -= move_speed
    if keys[pygame.K_s] and player1.bottom < HEIGHT // 2:
        player1.y += move_speed
    if keys[pygame.K_a] and player1.left > 0:
        player1.x -= move_speed
    if keys[pygame.K_d] and player1.right < WIDTH:
        player1.x += move_speed

    # Player 2 movement
    if keys[pygame.K_UP] and player2.top > 0:
        player2.y -= move_speed
    if keys[pygame.K_DOWN] and player2.bottom < HEIGHT // 2:
        player2.y += move_speed
    if keys[pygame.K_LEFT] and player2.left > 0:
        player2.x -= move_speed
    if keys[pygame.K_RIGHT] and player2.right < WIDTH:
        player2.x += move_speed

    # Check player collision
    if player1.colliderect(player2):
        player1.x, player1.y = old_pos1.x, old_pos1.y
        player2.x, player2.y = old_pos2.x, old_pos2.y

    # Check food collisions (if not trying to eat)
    for food in food_list:
        if player1.colliderect(food.rect) and not keys[pygame.K_e]:
            player1.x, player1.y = old_pos1.x, old_pos1.y
        if player2.colliderect(food.rect) and not keys[pygame.K_r]:
            player2.x, player2.y = old_pos2.x, old_pos2.y

def handle_steal():
    global player1_life, player2_life, steal_cooldown, steal_effect_timer
    
    if steal_cooldown > 0:
        steal_cooldown -= 1
        return

    keys = pygame.key.get_pressed()
    
    steal_range = pygame.Rect(player1.x - 10, player1.y - 10, 
                             PLAYER_WIDTH + 20, PLAYER_HEIGHT + 20)
    
    if steal_range.colliderect(player2):
        if keys[pygame.K_SPACE] and player2_life > 0:
            player2_life -= 1
            player1_life += 1
            steal_cooldown = STEAL_COOLDOWN_MAX
            steal_effect_timer = 15
            print("Player 1 stole a life! P1:", player1_life, "P2:", player2_life)
            
        if keys[pygame.K_RETURN] and player1_life > 0:
            player1_life -= 1
            player2_life += 1
            steal_cooldown = STEAL_COOLDOWN_MAX
            steal_effect_timer = 15
            print("Player 2 stole a life! P1:", player1_life, "P2:", player2_life)

def handle_food():
    global food_list, player1_life, player2_life
    
    keys = pygame.key.get_pressed()
    
    p1_food_detect = pygame.Rect(player1.x - 5, player1.y - 5, 
                                PLAYER_WIDTH + 10, PLAYER_HEIGHT + 10)
    p2_food_detect = pygame.Rect(player2.x - 5, player2.y - 5, 
                                PLAYER_WIDTH + 10, PLAYER_HEIGHT + 10)
    
    for food in food_list[:]:
        if p1_food_detect.colliderect(food.rect) and keys[pygame.K_e]:
            food_list.remove(food)
            player1_life += 1
            print("Player 1 ate food! Life:", player1_life)
            food_list.append(spawn_new_food())
            
        if p2_food_detect.colliderect(food.rect) and keys[pygame.K_r]:
            food_list.remove(food)
            player2_life += 1
            print("Player 2 ate food! Life:", player2_life)
            food_list.append(spawn_new_food())

# AI Movement Logic (Blue Player)
def blue_player_action(game_state):
    # Randomly choose one of the possible actions
    actions = ['w', 's', 'a', 'd', 'none']
    action = random.choice(actions)  # Replace with a more complex decision logic later
    return action

def ai_move():
    global player1
    game_state = update_matrix()  # Get current game state (matrix of ones and zeros)
    action = blue_player_action(game_state)  # Get AI action for current state

    # Apply AI action to blue player
    move_speed = 5
    if action == 'w' and player1.top > 0:
        player1.y -= move_speed
    if action == 's' and player1.bottom < HEIGHT // 2:
        player1.y += move_speed
    if action == 'a' and player1.left > 0:
        player1.x -= move_speed
    if action == 'd' and player1.right < WIDTH:
        player1.x += move_speed

# Game loop
clock = pygame.time.Clock()
running = True

# Add initial food
food_list = [spawn_new_food() for _ in range(5)]

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    handle_movement()
    handle_steal()
    handle_food()
    ai_move()  # Make AI move

    if steal_effect_timer > 0:
        steal_effect_timer -= 1
    
    game_matrix = update_matrix()
    draw_game()
    draw_matrix(game_matrix)

    if player1_life <= 0 or player2_life <= 0:
        running = False

    pygame.display.update()
    clock.tick(60)

pygame.quit()


Player 2 ate food! Life: 11
Player 1 stole a life! P1: 11 P2: 10
Player 1 stole a life! P1: 12 P2: 9
Player 2 stole a life! P1: 11 P2: 10


In [22]:
import pygame
import random

# Constants
GRID_SIZE = 20
MATRIX_WIDTH = 40
MATRIX_HEIGHT = 15
WIDTH, HEIGHT = 800, 600
MATRIX_HEIGHT_PX = HEIGHT // 2

# Initialize pygame
pygame.init()

# Screen setup
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Game and Matrix Window")

# Colors
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BRIGHT_GREEN = (150, 255, 150)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)

# Game objects
PLAYER_WIDTH, PLAYER_HEIGHT = 40, 40
FOOD_SIZE = 5
player1 = pygame.Rect(WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)
player2 = pygame.Rect(3 * WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)

# Food system
class Food:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, FOOD_SIZE, FOOD_SIZE)
        self.spawn_effect = 15  # Frames the spawn effect will last

food_list = [Food(random.randint(0, WIDTH - FOOD_SIZE), 
                 random.randint(0, HEIGHT // 2 - FOOD_SIZE)) for _ in range(5)]

# Player states
player1_life = 10
player2_life = 10
blue_last_state = None
blue_last_action = None
blue_last_reward = None
blue_experiences = []

# Fonts
font = pygame.font.Font(None, 36)

def spawn_new_food():
    return Food(random.randint(0, WIDTH - FOOD_SIZE),
                random.randint(0, HEIGHT // 2 - FOOD_SIZE))

def update_matrix():
    game_matrix = [[0] * MATRIX_WIDTH for _ in range(MATRIX_HEIGHT)]
    for player in [player1, player2]:
        for y in range(player.top // GRID_SIZE, (player.bottom) // GRID_SIZE):
            for x in range(player.left // GRID_SIZE, (player.right) // GRID_SIZE):
                if 0 <= y < MATRIX_HEIGHT and 0 <= x < MATRIX_WIDTH:
                    game_matrix[y][x] = 1
    for food in food_list:
        row, col = food.rect.top // GRID_SIZE, food.rect.left // GRID_SIZE
        if 0 <= row < MATRIX_HEIGHT and 0 <= col < MATRIX_WIDTH:
            game_matrix[row][col] = 1
    return game_matrix

def draw_matrix(game_matrix):
    matrix_surface = pygame.Surface((WIDTH, MATRIX_HEIGHT_PX))
    matrix_surface.fill(BLACK)
    for y, row in enumerate(game_matrix):
        row_text = " ".join(str(cell) for cell in row)
        text_surface = font.render(row_text, True, WHITE)
        matrix_surface.blit(text_surface, (10, y * 20))
    screen.blit(matrix_surface, (0, HEIGHT // 2))

def draw_game():
    screen.fill(BLACK)
    
    # Draw players
    pygame.draw.rect(screen, BLUE, player1)
    pygame.draw.rect(screen, RED, player2)
    
    # Draw food
    for food in food_list:
        pygame.draw.rect(screen, GREEN, food.rect)
    
    life_text1 = font.render(f"Player 1 Life: {player1_life}", True, WHITE)
    life_text2 = font.render(f"Player 2 Life: {player2_life}", True, WHITE)
    screen.blit(life_text1, (10, 10))
    screen.blit(life_text2, (WIDTH - 200, 10))

def handle_movement():
    global player2
    keys = pygame.key.get_pressed()
    move_speed = 5

    # Player 2 movement
    if keys[pygame.K_UP] and player2.top > 0:
        player2.y -= move_speed
    if keys[pygame.K_DOWN] and player2.bottom < HEIGHT // 2:
        player2.y += move_speed
    if keys[pygame.K_LEFT] and player2.left > 0:
        player2.x -= move_speed
    if keys[pygame.K_RIGHT] and player2.right < WIDTH:
        player2.x += move_speed

def blue_player_logic(game_matrix):
    global player1, blue_last_state, blue_last_action, blue_last_reward
    actions = ["UP", "DOWN", "LEFT", "RIGHT"]
    
    if blue_last_state is not None:
        reward = player1_life - blue_last_reward if blue_last_reward is not None else 0
        blue_experiences.append((blue_last_state, blue_last_action, reward))
        blue_last_reward = player1_life

    blue_last_state = [cell for row in game_matrix for cell in row]
    blue_last_action = random.choice(actions)
    
    if blue_last_action == "UP" and player1.top > 0:
        player1.y -= 5
    elif blue_last_action == "DOWN" and player1.bottom < HEIGHT // 2:
        player1.y += 5
    elif blue_last_action == "LEFT" and player1.left > 0:
        player1.x -= 5
    elif blue_last_action == "RIGHT" and player1.right < WIDTH:
        player1.x += 5

def handle_food():
    global food_list, player1_life, player2_life
    
    keys = pygame.key.get_pressed()
    
    for food in food_list[:]:
        if player1.colliderect(food.rect):
            food_list.remove(food)
            player1_life += 1
            food_list.append(spawn_new_food())
            
        if player2.colliderect(food.rect) and keys[pygame.K_r]:
            food_list.remove(food)
            player2_life += 1
            food_list.append(spawn_new_food())

def display_debug_info():
    """Display the blue player's state, action, and reward in real-time."""
    debug_area = pygame.Surface((WIDTH, 100))
    debug_area.fill((30, 30, 30))
    
    state_text = font.render(f"State: {blue_last_state[:5]}... (truncated)", True, WHITE)
    action_text = font.render(f"Action: {blue_last_action}", True, WHITE)
    reward_text = font.render(f"Reward: {blue_last_reward}", True, WHITE)
    
    debug_area.blit(state_text, (10, 10))
    debug_area.blit(action_text, (10, 40))
    debug_area.blit(reward_text, (10, 70))
    screen.blit(debug_area, (0, HEIGHT - 100))

# Game loop
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    handle_movement()
    game_matrix = update_matrix()
    blue_player_logic(game_matrix)
    handle_food()
    
    draw_game()
    draw_matrix(game_matrix)
    display_debug_info()

    pygame.display.update()
    clock.tick(60)

pygame.quit()

good
